In [20]:
import gzip
import json
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [21]:
brands = []
receipts = []
users = []

def load_json_lines(file_path, data_list):
    with gzip.open(file_path, 'rt', encoding='utf-8') as f:
        for line in f:
            line = line.strip() 
            if line: 
                try:
                    data_list.append(json.loads(line))
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON in {file_path}: {e} - Line: {line}")

load_json_lines('./data/brands.json.gz', brands)
load_json_lines('./data/receipts.json.gz', receipts)
load_json_lines('./data/users.json.gz', users)

Error decoding JSON in ./data/users.json.gz: Expecting value: line 1 column 1 (char 0) - Line: users.json                                                                                          0100777 0000000 0000000 00000254301 14567170355 010072  0                                                                                                    ustar 00                                                                                                                                                                                                                                                       {"_id":{"$oid":"5ff1e194b6a9d73a3a9f1052"},"active":true,"createdDate":{"$date":1609687444800},"lastLogin":{"$date":1609687537858},"role":"consumer","signUpSource":"Email","state":"WI"}
Error decoding JSON in ./data/users.json.gz: Expecting value: line 1 column 1 (char 0) - Line:                                                                                                                 

### Connecting Brands and ReceiptItems

An issue that I found is that there's no clear reference between `brands` and `receiptItems`. Therefore, it's important to find a unique key in `brands` that is also present in `receiptItems`

In [22]:
unique_barcodes = set()
duped_barcodes = []

for brand in brands:
    if brand['barcode'] in unique_barcodes:
        duped_barcodes.append(brand['barcode'])
        
    unique_barcodes.add(brand['barcode'])

In [23]:
print(f"The amount of brands: {len(brands)}")
print(f"The amount of unique barcodes: {len(unique_barcodes)}")

The amount of brands: 1167
The amount of unique barcodes: 1160


In [25]:
for barcode in duped_barcodes:
    print(f"Items with the same barcode: {barcode}")
    for brand in brands:
        if brand['barcode'] == barcode:
            print(f"- {brand['brandCode']} / {brand['name']}")
    print("--------------------")

Items with the same barcode: 511111504139
- CHRISXYZ / Chris Brand XYZ
- PACE / Pace
--------------------
Items with the same barcode: 511111504788
- TEST / test
- PIONEER WOMAN / The Pioneer Woman
--------------------
Items with the same barcode: 511111204923
- 0987654321 / Brand1
- CHESTERS / CHESTER'S
--------------------
Items with the same barcode: 511111305125
- CHRISIMAGE / Chris Image Test
- 511111305125 / Rachael Ray Everyday
--------------------
Items with the same barcode: 511111605058
- 511111605058 / Health Magazine
- 09090909090 / Brand2
--------------------
Items with the same barcode: 511111704140
- PREGO / Prego
- DIETCHRIS2 / Diet Chris Cola
--------------------
Items with the same barcode: 511111004790
- ALEXA / alexa
- BITTEN / Bitten Dressing
--------------------


Here, we can see that `barcode` is almost a unique key that we can use to connect `brands` and `receiptItems`.

We're not using `brandCode` because of its non-uniqueness issue. I understand it may be technically challenging to have `brand_id` attached to the `receiptItems` considering how it is reliant on the scanning technology, which may or may not fetch the accurate data from the backend.

In [29]:
receipt_items_with_barcode = []
receipt_items_without_barcode = []

for receipt in receipts:
    if 'rewardsReceiptItemList' in receipt:
        for item in receipt['rewardsReceiptItemList']:
            if 'barcode' in item or \
                'userFlaggedBarcode' in item or \
                'originalMetaBriteBarcode' in item:
                receipt_items_with_barcode.append(item)
            else:
                receipt_items_without_barcode.append(item)

print(f"The amount of receipt items with barcode: {len(receipt_items_with_barcode)}")
print(f"The amount of receipt items without barcode: {len(receipt_items_without_barcode)}")

The amount of receipt items with barcode: 3240
The amount of receipt items without barcode: 3701


The tradeoff of using `barcode` is that some `receiptItems` may not be parsed perfectly and thus missing that attribute. To be precise, more than half of the receipts are missing their `barcode`.

We can also argue to include `userFlaggedBarcode` and `originalMetaBriteBarcode` as further deep dives tell that those two attributes have the potential to replace the original `barcode`.

### Deep Dive into Attributes of ReceiptItems

In [62]:
receipts_finished = []
receipts_flagged = []
receipts_pending = []
receipts_submitted = []
receipts_rejected = []

for receipt in receipts:
    if 'rewardsReceiptStatus' in receipt:
        if receipt['rewardsReceiptStatus'] == 'FLAGGED':
            receipts_flagged.append(receipt)
        if receipt['rewardsReceiptStatus'] == 'FINISHED':
            receipts_finished.append(receipt)
        if receipt['rewardsReceiptStatus'] == 'SUBMITTED':
            receipts_submitted.append(receipt)
        if receipt['rewardsReceiptStatus'] == 'REJECTED':
            receipts_rejected.append(receipt)
        if receipt['rewardsReceiptStatus'] == 'PENDING':
            receipts_pending.append(receipt)

In [63]:
print(f"The amount of receipts flagged: {len(receipts_flagged)}")
print(f"The amount of receipts pending: {len(receipts_pending)}")
print(f"The amount of receipts finished: {len(receipts_finished)}")
print(f"The amount of receipts flagged: {len(receipts_flagged)}")
print(f"The amount of receipts rejected: {len(receipts_rejected)}")

The amount of receipts flagged: 46
The amount of receipts pending: 50
The amount of receipts finished: 518
The amount of receipts flagged: 46
The amount of receipts rejected: 71


### Issue of Aggregating Points and Spendings

Using the relational design that we have discussed in the first section of this assessment. To refresh, we're operating on `receiptItems` granularity and subsequently missing / removing some quantitative measures from the `receipts` level.

For example, there are"

1. `pointsEarned`
2. `totalSpent` 

in `receipts` objects but there are also:

1. `pointsEarned`
2. `finalPrice` or `itemPrice` 

in `receiptItems`. 

It's always a good idea to not muddle the fact table by combining two correlated attributes into one. Therefore, let's remove `pointsEarned` and `totalSpent`, and see if the aggregated correlated attributes in the `receiptItems` equal to the `receipts` level.

In [31]:
not_equal_spending = []

for receipt in receipts:
    spending = 0

    if 'totalSpent' not in receipt:
        continue

    if 'rewardsReceiptItemList' in receipt:
        for item in receipt['rewardsReceiptItemList']:
            if 'finalPrice' in item:
                spending = spending + float(item['finalPrice'])

    if 'totalSpent' in receipt:
        if spending != float(receipt['totalSpent']):
            not_equal_spending.append(receipt)

In [32]:
not_equal_points = []

for receipt in receipts:
    points = 0

    if 'pointsEarned' not in receipt:
        continue

    if 'rewardsReceiptItemList' in receipt:
        for item in receipt['rewardsReceiptItemList']:
            if 'pointsEarned' in item:
                points = points + float(item['pointsEarned'])

    if 'pointsEarned' in receipt:
        if points != float(receipt['pointsEarned']):
            not_equal_points.append(receipt)

In [34]:
print(f"The amount of receipts with total points aggregated doesn't match: {len(not_equal_points)}")
print(f"The amount of receipts with total spends aggregated doesn't match: {len(not_equal_spending)}")

The amount of receipts with total points aggregated doesn't match: 548
The amount of receipts with total spends aggregated doesn't match: 55


Since `pointsEarned` is not the main focus of the business questions, therefore, we can ignore it for now despite having a large amount of unequal aggregates.

From the quick analysis, above, we see that there's a total of 55 receipts with total spending doesn't match the aggregated `finalPrice` in their corresponding `receiptItems`. Because of the low amount, let's go on with the removal of `receipts` level measures.

### Issue of Points and Price Missing

In [50]:
missing_price_items = []
missing_points_items = []
missing_points_receipts = []
price_measures_diff = []

for receipt in receipts:
    if 'pointsEarned' in receipt:
        missing_points_receipts.append(receipt)

    if 'rewardsReceiptItemList' in receipt:
        for item in receipt['rewardsReceiptItemList']:
            if 'finalPrice' in item and 'itemPrice' in item:
                if item['finalPrice'] != item['itemPrice']:
                    price_measures_diff.append(item)

            if 'finalPrice' not in item:
                missing_price_items.append(item)
            if 'pointsEarned' not in item:
                missing_points_items.append(item)

In [49]:
print(f"The amount of receipt items without price: {len(missing_price_items)}")
print(f"The amount of receipt items without points: {len(missing_points_items)}")

The amount of receipt items without price: 174
The amount of receipt items without points: 6014


This is a further motivation to ignore points as of now. Moreover, there's only 174 receipt items without `finalPrice` attached to them, which is significantly low in proportion.

In [51]:
print(f"The amount of receipt items without points: {len(missing_points_receipts)}")

The amount of receipt items without points: 609


The amount of receipts without `pointsEarned` is significantly lower than their corresponding `receiptItems` and therefore if we're interested in studying points metric, we can use this measure on the `receipts` level.

### Issue of Different Price Measures

In [52]:
print(f"The amount of receipt items with different price measures: {len(price_measures_diff)}")

The amount of receipt items with different price measures: 4


The amount of receipt items with different `finalPrice` and `itemPrice` is very low. Therefore, we can somewhat disregard this issue as a technical fluke and use only one measure in the OLAP.